<a href="https://colab.research.google.com/github/Hrithik2212/-RNN-Sentimental-Analysis-on-Movie-Reviews---IMDB/blob/master/08_Langchain_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Intro to LangChain

LangChain is a popular framework that allow users to quickly build apps and pipelines around **L**arge **L**anguage **M**odels. It can be used to for chatbots, **G**enerative **Q**uestion-**A**nwering (GQA), summarization, and much more.

The core idea of the library is that we can _"chain"_ together different components to create more advanced use-cases around LLMs. Chains may consist of multiple components from several modules:

* **Prompt templates**: Prompt templates are, well, templates for different types of prompts. Like "chatbot" style templates, ELI5 question-answering, etc

* **LLMs**: Large language models like GPT-3, BLOOM, etc

* **Agents**: Agents use LLMs to decide what actions should be taken, tools like web search or calculators can be used, and all packaged into logical loop of operations.

* **Memory**: Short-term memory, long-term memory.

In [1]:
!pip install -qU langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


# Using LLMs in LangChain

LangChain supports several LLM providers, like Hugging Face and OpenAI.

Let's start our exploration of LangChain by learning how to use a few of these different LLM integrations.

## Hugging Face

We first need to install additional prerequisite libraries:

In [2]:
!pip install -qU huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 3.0 MB/s eta 0:00:00


For Hugging Face models we need a Hugging Face Hub API token. We can find this by first getting an account at [HuggingFace.co](https://huggingface.co/) and clicking on our profile in the top-right corner > click *Settings* > click *Access Tokens* > click *New Token* > set *Role* to *write* > *Generate* > copy and paste the token below:

In [3]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''  # put your token here

In [7]:
from pprint import pprint as pp

We can then generate text using a HF Hub model (we'll use `google/flan-t5-x1`) using the Inference API built into Hugging Face Hub.

_(The default Inference API doesn't use specialized hardware and so can be slow and cannot run larger models like `bigscience/bloom-560m` or `google/flan-t5-xxl`)_

In [13]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

# initialize HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="bigscience/bloom-560m",
    model_kwargs={"temperature":1e-9}
)

# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5
)

question = "Which NFL team won the Super Bowl in the 2010 season?"

print(llm_chain.run(question))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


 The Cowboys.


If we'd like to ask multiple questions we can by passing a list of dictionary objects, where the dictionaries must contain the input variable set in our prompt template (`"question"`) that is mapped to the question we'd like to ask.

In [15]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.generate(qs)
pp(res)

LLMResult(generations=[[Generation(text=' The Cowboys.')], [Generation(text=' 6')], [Generation(text=' The 12th person on the moon was')], [Generation(text=' The blade of grass has a')]], llm_output=None, run=[RunInfo(run_id=UUID('74e5f369-25e0-45c8-9127-377959889989')), RunInfo(run_id=UUID('63d0ab38-7535-4294-b810-9d87da635dd2')), RunInfo(run_id=UUID('261e7fea-bb4c-4de6-8d43-11bc27603c9b')), RunInfo(run_id=UUID('ee4c9135-44e5-4d29-91bb-63b652d58f43'))])


It is a LLM, so we can try feeding in all questions at once:

In [16]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

The


But with this model it doesn't work too well, we'll see this approach works better with different models soon.

## OpenAI

Start by installing additional prerequisites:

In [19]:
!pip install -qU openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00


We can also use OpenAI's generative models. The process is similar, we need to
give our API key which can be retrieved by signing up for an account on the
[OpenAI website](https://openai.com/api/) (see top-right of page). We then pass the API key below:

In [17]:
import os

os.environ['OPENAI_API_KEY'] = ''

If using OpenAI via Azure you should also set:

```python
os.environ['OPENAI_API_TYPE'] = 'azure'
# API version to use (Azure has several)
os.environ['OPENAI_API_VERSION'] = '2022-12-01'
# base URL for your Azure OpenAI resource
os.environ['OPENAI_API_BASE'] = 'https://your-resource-name.openai.azure.com'
```

Then we decide on which model we'd like to use, there are several options but we will go with `text-davinci-003`:

In [20]:
from langchain.llms import OpenAI

davinci = OpenAI(model_name='text-davinci-003')

Alternatively if using Azure OpenAI we do:

```python
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name="your-azure-deployment",
    model_name="text-davinci-003"
)
```

We'll use the same simple question-answer prompt template as before with the Hugging Face example. The only change is that we now pass our OpenAI LLM `davinci`:

In [21]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(llm_chain.run(question))

 The Green Bay Packers won the Super Bowl in the 2010 season.


The same works again for multiple questions using `generate`:

In [23]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_chain.generate(qs)

LLMResult(generations=[[Generation(text=' The Green Bay Packers won the Super Bowl in the 2010 season.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' 193.04 centimeters', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' Alan Bean was the 12th person on the moon. He was part of the Apollo 12 mission in 1969.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' A blade of grass does not have any eyes.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 75, 'total_tokens': 124}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('6ca888c2-3b8c-49fd-bdc3-5700a9f30813')), RunInfo(run_id=UUID('df409dcc-a095-4e7e-8587-a25a1226b3d1')), RunInfo(run_id=UUID('9356e192-f7a2-4808-bae1-aeeb02971078')), RunInfo(run_id=UUID('dae76324-4dc8-4949-83b2-f6f3679d5b9d'))])

Note that the below format doesn't feed the questions in iteratively but instead all in one chunk.

In [24]:
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
print(llm_chain.run(qs))


1. The New Orleans Saints 
2. 193 centimeters 
3. Harrison Schmitt 
4. Zero eyes.


Now we can try to answer all question in one go, as mentioned, more powerful LLMs like `text-davinci-003` will be more likely to handle these more complex queries.

In [25]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

The Green Bay Packers won the Super Bowl in the 2010 season.
If you are 6 ft 4 inches, you are 193.04 centimeters tall.
The 12th person on the moon was Alan L. Bean.
A blade of grass has no eyes.
